In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
#! pip install lxml
#! pip install unidecode
#! pip install selenium

In [3]:
#! pip install bs4

In [4]:
#! pip install webdriver_manager

In [5]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt 
import chardet
import geopandas as gpd

## Import "Data Dengue infected by district for 2015 to 2021"

In [67]:
base = open(r'../../_data/data_dengue_peru.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']
charenc

'UTF-8-SIG'

In [68]:
table1 = pd.read_csv('../../_data/data_dengue_peru.csv', encoding = charenc, thousands = ",", dtype= {"Ubigeo":"str"})
table1

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
3,2020,1,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,0.0
4,2020,53,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0
...,...,...,...,...,...,...,...,...
172139,2021,49,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN
172140,2021,49,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN
172141,2021,50,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN
172142,2021,50,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN


In [69]:
table1.dtypes

Año                  int64
Semana               int64
Eventos o daños     object
Departamento        object
Distrito            object
Provincia           object
Ubigeo              object
Casos              float64
dtype: object

In [70]:
table1 ['Año'].unique()

array([2020, 2021, 2019, 2017, 2016, 2018, 2015], dtype=int64)

## Generate "Ubigeo" for Departments and Provinces, taking the first two and four numbers 

In [71]:
table1['Ubigeo_Departments'] = table1['Ubigeo'].astype(str).str[:2]
table1

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,Ubigeo_Departments
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0,08
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18
3,2020,1,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,0.0,08
4,2020,53,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0,08
...,...,...,...,...,...,...,...,...,...
172139,2021,49,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN,17
172140,2021,49,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN,17
172141,2021,50,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN,17
172142,2021,50,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN,17


In [72]:
table1['Ubigeo_Provinces'] = table1['Ubigeo'].astype(str).str[:4]
table1

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,Ubigeo_Departments,Ubigeo_Provinces
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0,08,0809
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18,1801
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,18,1801
3,2020,1,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,0.0,08,0809
4,2020,53,Dengue,CUSCO,MEGANTONI,LA CONVENCION,080914,2.0,08,0809
...,...,...,...,...,...,...,...,...,...,...
172139,2021,49,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN,17,1702
172140,2021,49,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN,17,1703
172141,2021,50,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN,17,1702
172142,2021,50,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN,17,1703


In [73]:
table1.dtypes

Año                     int64
Semana                  int64
Eventos o daños        object
Departamento           object
Distrito               object
Provincia              object
Ubigeo                 object
Casos                 float64
Ubigeo_Departments     object
Ubigeo_Provinces       object
dtype: object

In [26]:
table1 ['Casos'].unique()

array([2.0, nan, 0.0, 7.0, 5.0, 11.0, 10.0, 9.0, 27.0, 82.0, 28.0, 50.0,
       15.0, 13.0, 17.0, 18.0, 19.0, 63.0, 87.0, 141.0, 12.0, 164.0, 29.0,
       118.0, 79.0, 81.0, 40.0, 31.0, 20.0, 6.0, 4.0, 8.0, 89.0, 46.0,
       86.0, 76.0, 300.0, 97.0, 444.0, 21.0, 1.0, 75.0, 418.0, 14.0,
       320.0, 52.0, 54.0, 130.0, 77.0, 102.0, 22.0, 67.0, 26.0, 36.0,
       32.0, 25.0, 56.0, 23.0, 33.0, 16.0, 3.0, 42.0, 34.0, 64.0, 70.0,
       71.0, 24.0, 49.0, 41.0, 91.0, 109.0, 43.0, 37.0, 94.0, 113.0, 48.0,
       38.0, 45.0, 62.0, 119.0, 39.0, 107.0, 88.0, 132.0, 144.0, 101.0,
       186.0, 57.0, 60.0, 44.0, 53.0, 69.0, 47.0, 55.0, 35.0, 72.0, 93.0,
       134.0, 84.0, 80.0, 95.0, 194.0, 155.0, 250.0, 74.0, 61.0, 166.0,
       198.0, 256.0, 215.0, 30.0, 66.0, 65.0, 105.0, 241.0, 96.0, 123.0,
       78.0, 133.0, 59.0, 68.0, 73.0, 51.0, 83.0, 137.0, 100.0, 111.0,
       185.0, 221.0, 248.0, 199.0, 174.0, 152.0, 163.0, 197.0, 169.0,
       255.0, 142.0, 291.0, 99.0, 165.0, 136.0, 122.0, 176.0, 1

In [74]:
table_dengue['Ubigeo'].unique().size

475

## Geopandas: Plot the number of cases in 2021 by district, usign continuous legend. 

In [75]:
table2 = gpd.read_file( r'..\..\_data\LIMITE_DISTRITAL_2020_INEI\INEI_LIMITE_DISTRITAL.shp' )
table2

,CCDD,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,CNT_CCPP,DESCRIPCIO,DESCARGAR,WHATSAPP,CONTACTO,geometry
0,06,CAJAMARCA,05,CONTUMAZA,04,GUZMANGO,GUZMANGO,060504,0605,060504,50,GUZMANGO,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-78.91578 -7.32085, -78.91370 -7.322..."
1,06,CAJAMARCA,11,SAN MIGUEL,05,EL PRADO,EL PRADO,061105,0611,061105,15,EL PRADO,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-79.01679 -6.96945, -79.01581 -6.969..."
2,06,CAJAMARCA,11,SAN MIGUEL,09,NIEPOS,NIEPOS,061109,0611,061109,41,NIEPOS,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-79.21691 -6.85171, -79.21576 -6.851..."
3,06,CAJAMARCA,11,SAN MIGUEL,10,SAN GREGORIO,SAN GREGORIO,061110,0611,061110,48,SAN GREGORIO,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-79.08233 -7.00544, -79.08041 -7.005..."
4,06,CAJAMARCA,12,SAN PABLO,03,SAN LUIS,SAN LUIS GRANDE,061203,0612,061203,9,SAN LUIS,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-78.87623 -7.09747, -78.87471 -7.098..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,04,AREQUIPA,04,CASTILLA,04,CHACHAS,CHACHAS,040404,0404,040404,19,CHACHAS,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-71.95148 -14.83836, -71.95157 -14.8..."
1870,04,AREQUIPA,04,CASTILLA,05,CHILCAYMARCA,CHILCAYMARCA,040405,0404,040405,24,CHILCAYMARCA,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-72.37490 -15.24990, -72.37494 -15.2..."
1871,04,AREQUIPA,04,CASTILLA,06,CHOCO,CHOCO,040406,0404,040406,58,CHOCO,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-71.96976 -15.13597, -71.96952 -15.1..."
1872,04,AREQUIPA,04,CASTILLA,07,HUANCARQUI,HUANCARQUI,040407,0404,040407,25,HUANCARQUI,www.geogpsperu.com,931381206,juan.suyo@geogpsperu.com,"POLYGON ((-72.31258 -15.81426, -72.31211 -15.8..."


In [38]:
table2 ['NOMBDEP'].unique()

array(['CAJAMARCA', 'LAMBAYEQUE', 'LIMA', 'ANCASH', 'AREQUIPA', 'CUSCO',
       'AYACUCHO', 'HUANCAVELICA', 'APURIMAC', 'PIURA', 'TACNA', 'JUNIN',
       'HUANUCO', 'UCAYALI', 'MADRE DE DIOS', 'PUNO', 'LA LIBERTAD',
       'TUMBES', 'CALLAO', 'AMAZONAS', 'LORETO', 'SAN MARTIN', 'ICA',
       'PASCO', 'MOQUEGUA'], dtype=object)

In [76]:
table2.dtypes

CCDD            object
NOMBDEP         object
CCPP            object
NOMBPROV        object
CCDI            object
NOMBDIST        object
CAPITAL         object
UBIGEO          object
IDPROV          object
CODIGO          object
CNT_CCPP         int64
DESCRIPCIO      object
DESCARGAR       object
WHATSAPP        object
CONTACTO        object
geometry      geometry
dtype: object

In [77]:
dist_shp['UBIGEO'].unique().size

1874

In [41]:
## Corregir: los dos ubigeos no cuadran (UBIGEO y ubigeo)
dengue = pd.concat([dist_shp, table_dengue], axis=1)
dengue

,CCDD,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,...,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,Ubigeo_Departments,Ubigeo_Provinces,Ubigeo_Distrito
1159,01,AMAZONAS,01,CHACHAPOYAS,01,CHACHAPOYAS,CHACHAPOYAS,010101,0101,010101,...,51,Dengue,AYACUCHO,SIVIA,HUANTA,50407,NaN,50,5040,50407
1331,01,AMAZONAS,01,CHACHAPOYAS,02,ASUNCION,ASUNCION,010102,0101,010102,...,41,Dengue,PIURA,SECHURA,SECHURA,200801,NaN,20,2008,200801
1125,01,AMAZONAS,01,CHACHAPOYAS,03,BALSAS,BALSAS,010103,0101,010103,...,51,Dengue,PIURA,COLAN,PAITA,200504,NaN,20,2005,200504
1153,01,AMAZONAS,01,CHACHAPOYAS,04,CHETO,CHETO,010104,0101,010104,...,45,Dengue,AYACUCHO,SIVIA,HUANTA,50407,NaN,50,5040,50407
1172,01,AMAZONAS,01,CHACHAPOYAS,05,CHILIQUIN,CHILIQUIN,010105,0101,010105,...,17,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,0.0,80,8091,80914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17,Dengue,CUSCO,CAMANTI,QUISPICANCHI,81203,1.0,81,8120,81203
156920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,Dengue,CUSCO,CAMANTI,QUISPICANCHI,81203,1.0,81,8120,81203
156918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9,Dengue,CUSCO,CAMANTI,QUISPICANCHI,81203,1.0,81,8120,81203
156915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,Dengue,CUSCO,CAMANTI,QUISPICANCHI,81203,1.0,81,8120,81203


In [78]:
dengue_shp =dist_shp.merge(table_dengue, how='left', left_on=['UBIGEO'], right_on=['Ubigeo'])
dengue_shp

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [46]:
dengue_shp.dtypes

CCDD                    object
NOMBDEP                 object
CCPP                    object
NOMBPROV                object
CCDI                    object
NOMBDIST                object
CAPITAL                 object
UBIGEO                  object
IDPROV                  object
CODIGO                  object
CNT_CCPP               float64
DESCRIPCIO              object
DESCARGAR               object
WHATSAPP                object
CONTACTO                object
geometry              geometry
Año                      int64
Semana                   int64
Eventos o daños         object
Departamento            object
Distrito                object
Provincia               object
Ubigeo                   int64
Casos                   object
Ubigeo_Departments      object
Ubigeo_Provinces        object
Ubigeo_Distrito         object
dtype: object

In [50]:
dengue_shp['Casos'] = pd.to_numeric(dengue_shp['Casos'])
dengue_shp.dtypes

ValueError: Unable to parse string "1,364" at position 86434